In [3]:
from nba_api.stats.endpoints import LeagueGameLog
import pandas
import numpy

In [ ]:
season = '2023-24'
game_log = LeagueGameLog(season=season, season_type_all_star='Regular Season').get_data_frames()[0]

In [ ]:
file 
play_by_play_df = pd.read_csv()